In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *

# Created Graph class where vertex = flow and edge indicates contention between 2 particular flows

In [2]:
class Graph:
    def __init__(self, vertices):
        self.adjacency_list = {}
        self.edge_list = set()
        
        for node in range(1, vertices):
            self.adjacency_list[node] = set()

    def add_edge(self, node1, node2):
        self.adjacency_list[node1].add(node2)
        self.adjacency_list[node2].add(node1)
        self.edge_list.add((node1,node2))
        self.edge_list.add((node2,node1))

    def remove_edge(self, node1, node2):
        if node1 in self.adjacency_list and node2 in self.adjacency_list:
            if node2 in self.adjacency_list[node1]:
                self.adjacency_list[node1].remove(node2)
                self.adjacency_list[node2].remove(node1)
                
        self.edge_list.remove((node1,node2))
        self.edge_list.remove((node2,node1))

    def get_nodes_by_degree(self):
        nodes_with_degrees = [(node, len(neighbors)) for node, neighbors in self.adjacency_list.items()]
        return set([node for node, _ in sorted(nodes_with_degrees, key = lambda x: x[1])])

    def is_edge(self, node1, node2):
        return (node1, node2) in self.edge_list and (node2, node1) in self.edge_list

### This function checks whether there is contention happening between 2 flows for concurrent transmissions

In [3]:
def check_create_edge(flow1, flow2):
    flow1_sender = flow1[2]
    flow1_receiver = flow1[3]
    flow2_sender = flow2[2]
    flow2_receiver = flow2[3]

    if flow1_sender == flow2_sender or flow1_receiver == flow2_receiver: # 2 flows conflicting due to FD mode, therefore contention definitely happens
        return True
    # Contention occurs only if relative interference (RI) between 2 flows exceeds defined threshold value
    elif flow1_sender == flow2_receiver and flow2_sender == flow1_receiver: 
        return calculate_relative_intereference1() > interference_threshold
    elif flow1_sender == flow2_receiver:
        return max(calculate_relative_intereference1(), calculate_relative_intereference2(flow2_sender, flow1_receiver)) > interference_threshold
    elif flow2_sender == flow1_receiver:
        return max(calculate_relative_intereference1(), calculate_relative_intereference2(flow1_sender, flow2_receiver)) > interference_threshold
    else:
        return max(calculate_relative_intereference2(flow1_sender, flow2_receiver), calculate_relative_intereference2(flow2_sender, flow1_receiver)) > interference_threshold

# Generating dynamic contention graph between flows 

In [4]:
def flow_graph_generate():
    # Creating a graph 
    graph = Graph(len(flows))
    # Comparing flow pair at a time for graph connection
    for flow_index1 in range(1, len(flows)):
        flow_with_time1 = [flow_index1, 0.0]
        flow_with_time1 = str(flow_with_time1)
        
        if flow_with_time1 not in flows_candidate_relay_set_pairs:
            continue
            
        relay_type1, candidate_set1, random_candidate1 = flows_candidate_relay_set_pairs[flow_with_time1]
        
        # If flow 1 is not relayed OR there is no relaying candidate for flow, no need to include it in graph
        if relay_type1 == 'dont_relay' or random_candidate1 == -1:
            continue
            
        for flow_index2 in range(1, len(flows)):
            if flow_index1 == flow_index2: # Redundant because 2 flows are same so no need to calculate
                continue
                
            flow_with_time2 = [flow_index2, 0.0]
            flow_with_time2 = str(flow_with_time2)
            
            if flow_with_time2 not in flows_candidate_relay_set_pairs:
                continue
        
            relay_type2, candidate_set2, random_candidate2 = flows_candidate_relay_set_pairs[flow_with_time2]
            
            # If flow 2 is not relayed OR there is no relaying candidate for flow, no need to include it in graph
            if relay_type2 == 'dont_relay' or random_candidate2 == -1:
                continue
                
            # If random relaying candidate coincide for 2 flows OR there is contention between 2 flows, then connect those flows(nodes) via graph edge            
            if random_candidate1 == random_candidate2 or check_create_edge(flows[flow_index1],flows[flow_index2]):   
                graph.add_edge(flow_index1, flow_index2)

    return graph

In [5]:
Graph = flow_graph_generate()

In [6]:
Graph.get_nodes_by_degree()

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80}

## Code picked up from other paper: Reference number [20]

In [7]:
def generate_groups():
    vertices = Graph.get_nodes_by_degree()
    groups = []
    
    while(len(vertices)):
        group = set()
        
        for vertex in vertices:
            vertices_copy = vertices.copy()
            
            if not group:
                group.add(vertex)
                vertices_copy.remove(vertex)
            else:
                no_edge = True
                
                for included_vertex in group:
                    if(Graph.is_edge(vertex, included_vertex)):
                        no_edge = False
                        break
                        
                if(no_edge):
                    group.add(vertex)
                    vertices_copy.remove(vertex)
            
            vertices = vertices_copy
            
        groups.append(group)
        
    return groups

In [8]:
groups = generate_groups()

In [9]:
groups

[{1,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69},
 {2},
 {3},
 {4},
 {5},
 {6},
 {7},
 {8},
 {9},
 {10},
 {11},
 {12},
 {13},
 {14},
 {15},
 {16},
 {17},
 {18},
 {19},
 {20},
 {70},
 {71},
 {72},
 {73},
 {74},
 {75},
 {76},
 {77},
 {78},
 {79},
 {80},
 {21},
 {22},
 {23},
 {24},
 {25},
 {43},
 {44},
 {45},
 {46},
 {47},
 {48},
 {49},
 {50},
 {51},
 {52},
 {53},
 {26},
 {27},
 {54},
 {55}]